De Bruijn indices are numbers that disambiguate each shadowed variable inside lambdas.

Consider a curried function that shadows a variable:

```scala
{ (x : Int) => (x : Int) => 123 + x }   // Scala
```

The corresponding code in µDhall is:

     λ(x : Natural) → λ(x : Natural) → 123 + x

This code shadows the outer `x` inside the inner lambda. An equivalent code without shadowing is:

```scala
{ (x : Int) => (t : Int) => 123 + t }   // Scala
```

In µDhall:

     λ(x : Natural) → λ(t : Natural) → 123 + t

Here we just renamed the inner `x` to `t`. What if we do not want to rename `x` to `t` but still want to refer to the outer `x`?
In µDhall, we can write `x@1` for that:

     λ(x : Natural) → λ(x : Natural) → 123 + x@1

This code is equivalent to:

     λ(x : Natural) → λ(t : Natural) → 123 + x

The `1` in `x@1` is the de Bruijn index of the outer variable `x` when accessed from the inner scope.
De Bruijn indices are non-negative integers.

We usually do not write `x@0`, we write just `x`.

Each de Bruijn index points to a specific nested lambda in some outer scope for a variable with a given name. For example, in this code:

     λ(x : Natural) → λ(t : Natural) → λ(x : Natural) → 123 + x@1

the variable `x@1` still points to the outer `x`. The presence of another lambda with the argument `t` does not matter for counting the nested depth for `x`.

At top level, it is invalid to use an index that is greater than the number of nested lambdas. For example, these expressions are invalid at top level (as there cannot be any outer scope):

     λ(x : Natural) → 123 + x@1
     λ(x : Natural) → λ(x : Natural) → 123 + x@2

At top level, these expressions are just as invalid as the expression `123 + x` since we never defined `x`.

The variable `x` in the expression `123 + x` is considered **free**; meaning that it should be defined in the outer scope.
Similarly, `x@1` in `λ(x : Natural) → 123 + x@1` is free. It is invalid to have expressions with free variables at top level.
At the top level, all variables must be bound. Expressions with free variables must be within bodies of some functions that bind their free variables.

When a function of the form `λ x → ...` is applied to an argument, we will need to substitute the outer `x`. Then we may need to recalculate some de Bruijn indices.

Suppose we would like to evaluate a function applied to an argument in this code:

     ( (x : Natural) → λ(y : Natural) → λ(x : Natural) → x + x@1 + x@2 ) y

This expression has free variables `y` and `x@2`, so it can occur only within a function body that binds `x` and `y`. It is worth remarking that here we are about to evaluate an expression "under a lambda". That is, we are going to simplify the body of a function before applying that function.

To evaluate this expression correctly, we cannot simply substitute `y` instead of `x`. Instead:

- the outer `x` corresponds to `x@1` within the expression, so we need to substitute `y` instead of `x@1` while keeping `x` and `x@2` unchanged
- `y` is already bound in the inner scope; so, we need to write `y@1` instead of `y`, in order to refer to the free variable `y` in the outside scope
- as we remove the outer `x`, the free variable `x@2` will now become `x@1`.

So, we must evaluate the given expression like this:

     ( (x : Natural) → λ(y : Natural) → λ(x : Natural) → x + x@1 + x@2 ) y
       = 
     λ(y : Natural) → λ(x : Natural) → x + y@1 + x@1  

During a single substitution, sometimes we need to shift the index upwards by 1, and sometimes downwards by 1. Also note that the variable `x@0` was left unchanged: the decrementing of indices for `x` starts only at index `1` because we are in a scope under a `λ x`.

This motivates the general "shift" operation for working with de Bruijn indices. That operation is defined in the Dhall standard [here](https://github.com/dhall-lang/dhall-lang/blob/master/standard/shift.md) as a function of _four_ arguments:

     // d = +1 or d = -1: whether we will increment or decrement the indices
     // x is the variable symbol on which the indices will be shifted
     // m is the minimum index value for shifting; we will shift x@n only if n >= m
     // e1 is a target expression where we will shift all bound variables named `x`
     // e2 is the resulting expression
     
     shift(d, x, m, e1) = e2

The Dhall documentation uses the "proof notation" for all definitions. The proof notation shows how to prove a desired statement (or "judgment") by proving some other statements first, or by referring to axioms. The equation `shift(d, x, m, e1) = e2` is a judgment that the result of evaluating the `shift` function with some arguments happens to equal `e2`. But we prefer to interpret that equation as a recipe for computing `e2` given `d`, `x`, `m`, and `e1`.

Adapting the Dhall specification to µDhall, we write the following specification for `shift`, casing on each of the 9 expression types:

1. Variables get shifted if their index is above threshold:

       shift(d, x, m, x@n) = x@(n + d)  ; shift if n >= m
       shift(d, x, m, x@n) = x@n        ; no change if n < m
       shift(d, x, m, y@n) = y@n        ; no change if x and y are different symbols

2. All expressions other than lambdas, function types, and `let` expressions, will just recursively shift all their sub-expressions.
3. Lambdas and function types introduce a new bound variable, say `x`. If we are also shifting the symbol `x`, we will need to avoid shifting that new bound variable; so, we need to increment the threshold when descending into the body. Note that in `λ(x : T)` the variable `x` is not in scope for its type `T`; so we do not need to increment the threshold when shifting `T`. The specification looks like this:
    ```
    shift(d, x, m, A₀) = A₁   shift(d, x, m + 1, b₀) = b₁
    ─────────────────────────────────────────────────────
    shift(d, x, m, λ(x : A₀) → b₀) = λ(x : A₁) → b₁


    shift(d, x, m, A₀) = A₁     shift(d, x, m, b₀) = b₁
    ───────────────────────────────────────────────────  ; x ≠ y
    shift(d, x, m, λ(y : A₀) → b₀) = λ(y : A₁) → b₁


    shift(d, x, m, A₀) = A₁   shift(d, x, m + 1, b₀) = b₁
    ─────────────────────────────────────────────────────
    shift(d, x, m, ∀(x : A₀) → b₀) = ∀(x : A₁) → b₁


    shift(d, x, m, A₀) = A₁     shift(d, x, m, b₀) = b₁
    ───────────────────────────────────────────────────  ; x ≠ y
    shift(d, x, m, ∀(y : A₀) → b₀) = ∀(y : A₁) → b₁
    ```
    The last rule says that when we shift a symbol that is not bound, we just recursively shift all sub-expressions.
4. The `let` expressions also introduce a bound variable. Just like lambdas, `let x = a in b` needs to raise the shifting threshold when shifting `b` but not when shifting `a` because `x` is not in scope for `a`. (Recursive definitions are not supported!)
   ```
   shift(d, x, m, a₀) = a₁
   shift(d, x, m + 1, b₀) = b₁
   ───────────────────────────────────────────────────
   shift(d, x, m, let x = a₀ in b₀) = let x = a₁ in b₁


   shift(d, x, m, b₀) = b₁     shift(d, x, m, a₀) = a₁
   ───────────────────────────────────────────────────  ; x ≠ y
   shift(d, x, m, let y = a₀ in b₀) = let y = a₁ in b₁   
   ```
5. All other expressions just recursively shift all their sub-expressions. To make that code shorter, we implement a `map`-like function for the `Expr` type.

Here are the rules in mathematical notation:

$$
\frac{
  \begin{aligned}
    & \text{shift}(d, x, m, A_0) = A_1 \qquad \text{shift}(d, x, m + 1, b_0) = b_1
  \end{aligned}
}{
  \text{shift}(d, x, m, \lambda(x : A_0) \to b_0) = \lambda(x : A_1) \to b_1
}
$$

$$
\frac{
  \begin{aligned}
    & \text{shift}(d, x, m, A_0) = A_1 \qquad \text{shift}(d, x, m, b_0) = b_1
  \end{aligned}
}{
  \text{shift}(d, x, m, \lambda(y : A_0) \to b_0) = \lambda(y : A_1) \to b_1
}
; x \neq y
$$

$$
\frac{
  \begin{aligned}
    & \text{shift}(d, x, m, A_0) = A_1 \qquad \text{shift}(d, x, m + 1, b_0) = b_1
  \end{aligned}
}{
  \text{shift}(d, x, m, \forall(x : A_0) \to b_0) = \forall(x : A_1) \to b_1
}
$$

$$
\frac{
  \begin{aligned}
    & \text{shift}(d, x, m, A_0) = A_1 \qquad \text{shift}(d, x, m, b_0) = b_1
  \end{aligned}
}{
  \text{shift}(d, x, m, \forall(y : A_0) \to b_0) = \forall(y : A_1) \to b_1
}
; x \neq y
$$

$$
\frac{
  \begin{aligned}
    & \text{shift}(d, x, m, a_0) = a_1 \qquad \text{shift}(d, x, m + 1, b_0) = b_1
  \end{aligned}
}{
  \text{shift}(d, x, m, \text{let } x = a_0 \text{ in } b_0) = \text{let } x = a_1 \text{ in } b_1
}
$$

$$
\frac{
  \begin{aligned}
    & \text{shift}(d, x, m, b_0) = b_1 \qquad \text{shift}(d, x, m, a_0) = a_1
  \end{aligned}
}{
  \text{shift}(d, x, m, \text{let } y = a_0 \text{ in } b_0) = \text{let } y = a_1 \text{ in } b_1
}
; x \neq y
$$


Now we are ready to write the code for the `shift` function.

In [26]:
def shift(d: Int, x: String, m: Int, target: Expr): Expr = target match {
    case Expr.Variable(name, index) if name == x && index >= m     => Expr.Variable(name, index + d)  // Shifted.
    case Expr.NaturalLiteral(_) | Expr.Builtin(_) | Expr.Variable(_, _)   => target                   // Unchanged.
    case Expr.Lambda(name, tipe, body) if name == x                => Expr.Lambda(name, shift(d, x, m, tipe), shift(d, x, m + 1, body))
    case Expr.Forall(name, tipe, body) if name == x                => Expr.Forall(name, shift(d, x, m, tipe), shift(d, x, m + 1, body))
    case Expr.Let(name, subst, body)   if name == x                => Expr.Let(name, shift(d, x, m, subst), shift(d, x, m + 1, body))
    case _                                                         => target.map(shift(d, x, m, _)) 
}

cmd26.sc:3: The outer reference in this type test cannot be checked at run time.
        case Expr.NaturalLiteral(_) | Expr.Builtin(_) | Expr.Variable(_, _) => f(e)
                                ^
cmd26.sc:3: The outer reference in this type test cannot be checked at run time.
        case Expr.NaturalLiteral(_) | Expr.Builtin(_) | Expr.Variable(_, _) => f(e)
                                                  ^
cmd26.sc:3: The outer reference in this type test cannot be checked at run time.
        case Expr.NaturalLiteral(_) | Expr.Builtin(_) | Expr.Variable(_, _) => f(e)
                                                                     ^
cmd26.sc:4: The outer reference in this type test cannot be checked at run time.
        case Expr.Lambda(name, tipe, body)   => Expr.Lambda(name, tipe.map(f), body.map(f))
                        ^
cmd26.sc:5: The outer reference in this type test cannot be checked at run time.
        case Expr.Forall(name, tipe, body)   => Expr.Forall(name, ti

defined class ExprMap
defined function shift

In [36]:
val f1 = ("n".! :~ Natural.!) ~> ("n".! + 1.!)
expect(prettyprint(f1) == "λ(n : Natural) → n + 1")

val f2 = ("n".! :~ Natural.!) ~> ("p".! + 1.!)
expect(prettyprint(f2) == "λ(n : Natural) → p + 1")

Seq(
    shift(1, "x", 0, "x".!) -> "x".!!(1),
    shift(-1, "x", 0, "x".!!(1)) -> "x".!,
    shift(1, "x", 0, "x".! + 1.!) -> ("x".!!(1) + 1.!),
    shift(1, "x", 1, "x".!) -> "x".!,
    shift(1, "x", 0, "y".!) -> "y".!,
    shift(1, "n", 0, f1) -> f1,
    shift(1, "n", 0, f2) -> f2,
    shift(1, "p", 0, f1) -> f1,
    shift(1, "p", 0, f2) -> ("n".! :~ Natural.!) ~> ("p".!!(1) + 1.!),
).zipWithIndex.foreach { case ((e, expected), i) => expect(i >= 0 && e == expected) }

"Tests passed for shift()."

f1: Expr = Lambda(
  name = "n",
  tipe = Builtin(constant = Natural),
  body = BinaryOp(
    left = Variable(name = "n", index = 0),
    op = Plus,
    right = NaturalLiteral(value = 1)
  )
)
f2: Expr = Lambda(
  name = "n",
  tipe = Builtin(constant = Natural),
  body = BinaryOp(
    left = Variable(name = "p", index = 0),
    op = Plus,
    right = NaturalLiteral(value = 1)
  )
)
res36_5: String = "Tests passed for shift."

Using the `shift` function, we can implement safe substitution as a `subst` function. We define `sub(v -> s, body)` as a function that replaces a given free variable `v` by `s` in an expression `expr`. (Substitution ignores bound variables and only substitutes free variables.)

Here are some examples:

     sub(x -> 123, λ(x : Type) → x)   =   λ(x : Type) → x    // The variable `x` is not free under lambda.
     sub(x -> 123, λ(y : Type) → x)   =   λ(y : Type) → 123  // The variable `x` is free under lambda.

Substitution avoids introducing a name clash in variables by incrementing de Bruijn indices:

     subst(y -> x, λ(x : Type) → y) = λ(x : Type) → x@1

We see that `sub(v -> s, expr)` gives similar results to `let v = s in expr`.
But substitution is more general than `let` because we may use nonzero de Bruijn indices:

     sub(x@1 -> 123, λ(x : Type) → x@2)   =    λ(x : Type) → 123

Note here that `x@2` inside a `λ x` corresponds to `x@1` outside it. This is another example where `sub` must increment an index when going inside a lambda.

These examples motivate the specification of `sub`, which we adopt from [its specification in Dhall](https://github.com/dhall-lang/dhall-lang/blob/master/standard/substitution.md):

```
──────────────────────
sub(x@n -> e, x@n) = e


─────────────────────────  ; x@n ≠ y@p
sub(x@n -> e₀, y@p) = y@p


sub(x@n -> e₀, A₀) = A₁   shift(1, x, 0, e₀) = e₁   sub(x@(n+1) -> e₁, b₀) = b₁
───────────────────────────────────────────────────────────────────────────────
sub(x@n -> e₀, λ(x : A₀) → b₀) = λ(x : A₁) → b₁


sub(x@n -> e₀, A₀) = A₁   shift(1, y, 0, e₀) = e₁   sub(x@n -> e₁, b₀) = b₁
───────────────────────────────────────────────────────────────────────────  ; x ≠ y
sub(x@n -> e₀, λ(y : A₀) → b₀) = λ(y : A₁) → b₁


sub(x@n -> e₀, A₀) = A₁   shift(1, x, 0, e₀) = e₁   sub(x@(n+1) -> e₁, B₀) = B₁
───────────────────────────────────────────────────────────────────────────────
sub(x@n -> e₀, ∀(x : A₀) → B₀) = ∀(x : A₁) → B₁


sub(x@n -> e₀, A₀) = A₁   shift(1, y, 0, e₀) = e₁   sub(x@n -> e₁, B₀) = B₁
───────────────────────────────────────────────────────────────────────────  ; x ≠ y
sub(x@n -> e₀, ∀(y : A₀) → B₀) = ∀(y : A₁) → B₁


sub(x@n -> e₀, a₀) = a₁   shift(1, x, 0, e₀) = e₁   sub(x@(n+1) -> e₁, b₀) = b₁
───────────────────────────────────────────────────────────────────────────────
sub(x@n -> e₀, let x = a₀ in b₀) = let x = a₁ in b₁


sub(x@n -> e₀, a₀) = a₁   shift(1, y, 0, e₀) = e₁   sub(x@n -> e₁, b₀) = b₁
───────────────────────────────────────────────────────────────────────────  ; x ≠ y
sub(x@n -> e₀, let y = a₀ in b₀) = let y = a₁ in b₁
```

The evaluation of `sub(x@n -> s, e)` for all other `e` will just recursively substitute in all sub-expressions within `e`.

In the mathematical notation, these diagrams look like this:

$$
\frac{
}{
  \text{sub}(x\text{@}n \to e, x\text{@}n) = e
}
$$

$$
\frac{
}{
  \text{sub}(x\text{@}n \to e_0, y\text{@}p) = y\text{@}p
}
\quad\text{; If } x\text{@}n \neq y\text{@}p
$$

$$
\frac{
  \begin{aligned}
    & \text{sub}(x\text{@}n \to e_0, A_0) = A_1 \quad \text{shift}(1, x, 0, e_0) = e_1 \quad \text{sub}(x\text{@}(n+1) \to e_1, b_0) = b_1
  \end{aligned}
}{
  \text{sub}(x\text{@}n \to e_0, \lambda(x : A_0) \to b_0) = \lambda(x : A_1) \to b_1
}
$$

$$
\frac{
  \begin{aligned}
    & \text{sub}(x\text{@}n \to e_0, A_0) = A_1 \quad \text{shift}(1, y, 0, e_0) = e_1 \quad \text{sub}(x\text{@}n \to e_1, b_0) = b_1
  \end{aligned}
}{
  \text{sub}(x\text{@}n \to e_0, \lambda(y : A_0) \to b_0) = \lambda(y : A_1) \to b_1
}
\quad\text{; If } x \neq y
$$

$$
\frac{
  \begin{aligned}
    & \text{sub}(x\text{@}n \to e_0, A_0) = A_1 \quad \text{shift}(1, x, 0, e_0) = e_1 \quad \text{sub}(x\text{@}(n+1) \to e_1, B_0) = B_1
  \end{aligned}
}{
  \text{sub}(x\text{@}n \to e_0, \forall(x : A_0) \to B_0) = \forall(x : A_1) \to B_1
}
$$

$$
\frac{
  \begin{aligned}
    & \text{sub}(x\text{@}n \to e_0, A_0) = A_1 \quad \text{shift}(1, y, 0, e_0) = e_1 \quad \text{sub}(x\text{@}n \to e_1, B_0) = B_1
  \end{aligned}
}{
  \text{sub}(x\text{@}n \to e_0, \forall(y : A_0) \to B_0) = \forall(y : A_1) \to B_1
}
\quad\text{; If }  x \neq y
$$

$$
\frac{
  \begin{aligned}
    & \text{sub}(x\text{@}n \to e_0, a_0) = a_1 \quad \text{shift}(1, x, 0, e_0) = e_1 \quad \text{sub}(x\text{@}(n+1) \to e_1, b_0) = b_1
  \end{aligned}
}{
  \text{sub}(x\text{@}n \to e_0, \text{let } x = a_0 \text{ in } b_0) = \text{let } x = a_1 \text{ in } b_1
}
$$

$$
\frac{
  \begin{aligned}
    & \text{sub}(x\text{@}n \to e_0, a_0) = a_1 \quad \text{shift}(1, y, 0, e_0) = e_1 \quad \text{sub}(x\text{@}n \to e_1, b_0) = b_1
  \end{aligned}
}{
  \text{sub}(x\text{@}n \to e_0, \text{let } y = a_0 \text{ in } b_0) = \text{let } y = a_1 \text{ in } b_1
}
\quad\text{; If }  x \neq y
$$



We are now ready to implement `sub`.

In [74]:
def sub(v: Expr.Variable, s: Expr, target: Expr): Expr = {
  def getBodyAfterSubst(name: String, body: Expr): Expr = {
       val v1 = if (name == v.name) Expr.Variable(v.name, v.index + 1) else v
       val e1 = shift(1, name, 0, s)
       sub(v1, e1, body)
  }

  target match {
    case Expr.Variable(name, index) if name == v.name && index == v.index => s       // Substituted.
    case Expr.NaturalLiteral(_) | Expr.Builtin(_) | Expr.Variable(_, _)   => target  // Unchanged.
    case Expr.Lambda(name, tipe, body) => Expr.Lambda(name, sub(v, s, tipe), getBodyAfterSubst(name, body))
    case Expr.Forall(name, tipe, body) => Expr.Forall(name, sub(v, s, tipe), getBodyAfterSubst(name, body))
    case Expr.Let(name, subst, body)   => Expr.Let  (name, sub(v, s, subst), getBodyAfterSubst(name, body))
    case _  => target.map(sub(v, s, _)) 
  }
}

cmd74.sc:9: The outer reference in this type test cannot be checked at run time.
    case Expr.Variable(name, index) if name == v.name && index == v.index => s       // Substituted.
                      ^
cmd74.sc:10: The outer reference in this type test cannot be checked at run time.
    case Expr.NaturalLiteral(_) | Expr.Builtin(_) | Expr.Variable(_, _)   => target  // Unchanged.
                            ^
cmd74.sc:10: The outer reference in this type test cannot be checked at run time.
    case Expr.NaturalLiteral(_) | Expr.Builtin(_) | Expr.Variable(_, _)   => target  // Unchanged.
                                              ^
cmd74.sc:10: The outer reference in this type test cannot be checked at run time.
    case Expr.NaturalLiteral(_) | Expr.Builtin(_) | Expr.Variable(_, _)   => target  // Unchanged.
                                                                 ^
cmd74.sc:11: The outer reference in this type test cannot be checked at run time.
    case Expr.Lambda(nam

defined function sub

In [42]:
Seq(
    // Substitution has no effect on expressions without free variables.
    sub("x".!, 123.!, ("x".! :~ Natural.!) ~> ("x".! + 100.!))
        -> (("x".! :~ Natural.!) ~> ("x".! + 100.!)),

    // Substitution will replace free variables.
    sub("x".!, 123.!, ("y".! :~ Natural.!) ~> ("x".! + 100.!))
        -> (("y".! :~ Natural.!) ~> (123.! + 100.!)),

    // A variable can be still be free if its de Bruijn index is large enough.
    sub("x".!, 123.!, ("x".! :~ Natural.!) ~> ("x" !! 1))
        -> (("x".! :~ Natural.!) ~> 123.!),

    // Descending past a matching bound variable increments the index to substitute.
    sub("x" !! 1, 123.!, ("x".! :~ Natural.!) ~> ("x" !! 2))
        -> (("x".! :~ Natural.!) ~> 123.!),

    // Substitution must avoid creating a name clash ("variable capture").
    sub("y".!, "x".!, ("x".! :~ Natural.!) ~> "y".!)
        -> (("x".! :~ Natural.!) ~> ("x" !! 1)),

).zipWithIndex.foreach { case ((e, expected), i) => expect(i >= 0 && e == expected) }
"Tests passed for sub()."

res42_1: String = "Tests passed for sub()."